In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense , Input
from tensorflow.keras.losses import Loss
import numpy as np
print("Version of tensorflow is : ",tf.__version__)

Version of tensorflow is :  2.13.0


In [4]:
xs = np.arange(-5,5,1).astype('float32')
ys = xs * 2 - 1

huber loss function **without wrapper**

---

In that case we can not pass any type of parameter to the function, thresold is the most important parameter in that type of loss function , so we need to tune this.

In [3]:
def huber_loss(y_true , y_pred):
    thresold = 1
    error = y_true - y_pred
    return_type = tf.abs(error) <= thresold
    r1 = 0.5 * tf.square(error)
    r2 = thresold * (tf.abs(error) - (0.5*thresold))
    return tf.where(return_type , r1 , r2)

In [5]:
input = Input(shape=(1,) , name="input_layer")
output_layer = Dense(1 , name="output_layer")(input)
model_huber_loss = tf.keras.models.Model(inputs=input , outputs=output_layer)
model_huber_loss.compile(optimizer="sgd" , loss=huber_loss)
model_huber_loss.fit(xs,ys,epochs=500,verbose=0)
model_huber_loss.predict([[10.0]])

array([[19.019274]], dtype=float32)

huber loss function **with wrapper**

---

In that case we can pass any type of parameter to the function, thresold is the most important parameter in that type of loss function , so we need to tune this parameter.

In [7]:
def huber_loss_wrapper(thresold):
    def huber_loss(y_true , y_pred):
        error = y_true - y_pred
        return_type = tf.abs(error) <= thresold
        r1 = 0.5 * tf.square(error)
        r2 = thresold * (tf.abs(error) - (0.5*thresold))
        return tf.where(return_type , r1 , r2)
    return huber_loss

In [14]:
input = Input(shape=(1,) , name="input_layer")
output_layer = Dense(1 , name="output_layer")(input)
model_huber_loss_wrapper = tf.keras.models.Model(inputs=input , outputs=output_layer)
model_huber_loss_wrapper.compile(optimizer="sgd" , loss=huber_loss_wrapper(thresold=1))
model_huber_loss_wrapper.fit(xs,ys,epochs=500,verbose=0)
model_huber_loss_wrapper.predict([[10.0]])

array([[19.014534]], dtype=float32)

**Own loss Class & Object**

In [15]:
class Huber(Loss):
    thresold = 1
    def __init__(self , thresold):
        super().__init__()
        self.thresold = thresold
    def call(self , y_true , y_pred):
        error = y_true - y_pred
        return_type = tf.abs(error) <= self.thresold
        r1 = 0.5 * tf.square(error)
        r2 = self.thresold * (tf.abs(error) - (0.5*self.thresold))
        return tf.where(return_type , r1 , r2)

In [19]:
input = Input(shape=(1,) , name="input_layer")
output_layer = Dense(1 , name="output_layer")(input)
model_huber_loss_class = tf.keras.models.Model(inputs=input , outputs=output_layer)
model_huber_loss_class.compile(optimizer="sgd" , loss=Huber(thresold=1))
model_huber_loss_class.fit(xs,ys,epochs=500,verbose=0)
model_huber_loss_class.predict([[10.0]])

array([[19.020779]], dtype=float32)